In [ ]:
try:
    print(moviepy.__version)
except:
    from moviepy.editor import *
    import sys
    !{sys.executable} -m pip install moviepy
    !{sys.executable} -m pip install --upgrade moviepy

In [ ]:
import cv2
import os
import shutil
import pandas as pd
import numpy as np

In [ ]:
path_original_vids = 'Data/Original_data'
path_cutted_vids = 'Data/Cutted_videos_data'
path_pictures = 'Data/Picture_data'

In [ ]:
def make_data_folders(path_cutted_vids, path_pictures):
    """Deletes Picture_data directory if existig. Then create Picture_data directory.
    
    @type path_to: str
    @params path_to: Name of directorypath where data will be stored after manipulating.
    """
    try:
        shutil.rmtree(f'{path_cutted_vids}'), shutil.rmtree(f'{path_pictures}')

    except:
        pass
    
    os.mkdir(f'{path_cutted_vids}')
    os.mkdir(f'{path_pictures}')

    os.mkdir(f"{path_pictures}/PT")
    os.mkdir(f"{path_pictures}/NT")
    os.mkdir(f"{path_pictures}/PL")
    os.mkdir(f"{path_pictures}/NL")

In [ ]:
def crop_center(img, crop_x=300, crop_y=350):
    """Crops x and y from center.
    
    @type img: numpy.ndarray
    @param img: picture of subject
    @type crop_x: int
    @param crop_x: the width of the cropped picture
    @type crop_y: int
    @param crop_y: the height of the cropped picture
    """
    y,x,_ = img.shape
    start_x = x // 2 - crop_x//2
    start_y = y // 2 - crop_y//2
    return img[start_y:start_y+crop_y, start_x:start_x+crop_x, :]

In [ ]:
def get_video_samples(path_original_vids, path_cutted_vids): 
    """ Cuts the video's based on given start and endtime out of an excel file.
    
    @type path_original: str
    @param path_original: The path where all original videofiles are stored.
    @type path_cutted_vids: str
    @param path_cutted_vids: The path where all cutted videofiles will be stored.
    """
    df = pd.read_excel(r'Data/start_end_times_sample.xlsx')
    df['Name'] = df['Name']+df['Truth/Lie']
    df.drop('Truth/Lie', axis=1, inplace=True)
    
    print('getting names and time from xlsx file....')
    get_vid_sample = lambda row :[row[0]] + [(begin_time,end_time) for begin_time, end_time in zip(row[1::2],row[2::2])]
    movie_name_and_times = [get_vid_sample(row) for _, row in df.iterrows()]
    print('names and times gatherd')
    
    print('start loading in VideoFileClips')
    videos = [VideoFileClip(f'{path_original_vids}/{row[0]}.wmv') for row in movie_name_and_times]
    print('videos ready for converting')
    
    print('cutting the videos')
    cut_vid = lambda vid, begin, end : vid.subclip(int(begin), int(end))
    cutted_clips = [(times[0], cut_vid(video, time_x[0], time_x[1])) for video, times in zip(videos, movie_name_and_times) for time_x in times[1:] if not pd.isnull(time_x[0])]
    [cutted_clip[1].to_videofile(f'{path_cutted_vids}/{cutted_clip[0]}{index}.mp4', codec='libx264', temp_audiofile=f'Data/{cutted_clip[0]}{index}TEMP_MPY_wvf_snd.mp3', remove_temp=True) for index, cutted_clip in enumerate(cutted_clips)]
    print('Done!')

In [ ]:
def convert_video_picture(path_cutted_vids, path_pictures):
    """
    Converting video's into pictures and stores them.
    
    @type path_from: str
    @params path_from: Name of directorypath where data will be fetched from.
    @type path_to: str
    @params path_to: Name of directorypath where data will be stored after manipulating.
    
    """
    for file in os.listdir(f"{path_from}"):
        video_samp = cv2.VideoCapture(f"{path_from}/{file}")
        succes, image = video_samp.read()
        count = 0
        while succes:
            cropped_img = crop_center(image)print('getting names and time from xlsx file....')
            cv2.imwrite(f"{path_to}/{file[7:9]}/{file[:5]}frame{count}.jpg", cropped_img)
            succes, image = video_samp.read()
            count+=1

In [ ]:
make_data_folders(path_cutted_vids, path_pictures)
get_video_samples(path_original_vids, path_cutted_vids)
convert_video_picture(path_cutted_vids, path_pictures)

In [ ]:
def get_video_samples_iterative():     
    df = pd.read_excel(r'Data/start_end_times_sample.xlsx')

    df['Name'] = df['Name']+df['Truth/Lie']
    df.drop('Truth/Lie', axis=1, inplace=True)  
    
    get_vid_sample = lambda row :[row[0]] + [(begin_time,end_time) for begin_time, end_time in zip(row[1::2],row[2::2])]
    
    for _ , row in df.iterrows():
        get_vid_samp_times = get_vid_sample(row)
        index = 0
        
        for video_times in get_vid_samp_times[1:]:
            if not pd.isnull(video_times[0]):
                
                clip = VideoFileClip(f'{path_from}/{get_vid_samp_times[0]}.wmv')
                clip = clip.cutout(video_times[0],video_times[1])
                temp_audio_file = f'Data/{get_vid_samp_times[0]}{index}TEMP_MPY_wvf_snd.mp3'
                clip.to_videofile(f'./Data/{get_vid_samp_times[0]}{index}.mp4', codec='libx264', temp_audiofile=temp_audio_file, remove_temp=True)
                index+=1